In [5]:
# !pip install NLTK bert_score
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hdu5/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import csv
import random
import pandas as pd

from nltk import tokenize

from bert_score import BERTScorer

random.seed(42)

threshold = 0.55
#model_type = "facebook/bart-large"
model_type = "microsoft/deberta-xlarge-mnli"
model_lang = "en"
# load scorer
bert_scorer = BERTScorer(model_type=model_type, lang=model_lang)

Some weights of the model checkpoint at microsoft/deberta-xlarge-mnli were not used when initializing DebertaModel: ['pooler.dense.weight', 'classifier.bias', 'pooler.dense.bias', 'classifier.weight']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
with open('./AutoReview_summarized.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    # skip the header (pid, document, summary(feedback))
    next(csv_reader)
    pid_list, doc_list, sent_list = [], [], []
    for row in csv_reader:
        pid, doc, fdbk = row[0], row[1], row[2]
        pid_list.append(pid)
        doc_list.append(doc)
        sent_list.append(tokenize.sent_tokenize(fdbk))
        assert len(pid_list) == len(doc_list) == len(sent_list)
    num_reports = len(pid_list)

In [8]:
pid_out, doc_out, sent_out, label_out = [], [], [], []
neg_set = set()

for i, (pid, doc, sents) in enumerate(zip(pid_list, doc_list, sent_list)):
    print(f'Processing the report {i}.')
    for sent in sents:
        if len(sent)>4:
            for _ in range(100):
                random_idx = random.choice(list(range(0, i)) + list(range(i+1, num_reports)))
                rand_neg = random.choice(sent_list[random_idx])
                if len(rand_neg) > 4 and rand_neg not in neg_set:

                    P, R, F1 = bert_scorer.score(sents, [rand_neg]*len(sents))

                    if all(f1<0.55 for f1 in F1):
                        neg_set.add(rand_neg)

                        pid_out.append(pid)
                        doc_out.append(doc)
                        sent_out.append(sent)
                        label_out.append(1)

                        pid_out.append(pid)
                        doc_out.append(doc)
                        sent_out.append(rand_neg)
                        label_out.append(0)    
                        break

Processing the report 0.
Processing the report 1.
Processing the report 2.
Processing the report 3.
Processing the report 4.
Processing the report 5.
Processing the report 6.
Processing the report 7.
Processing the report 8.
Processing the report 9.
Processing the report 10.
Processing the report 11.
Processing the report 12.
Processing the report 13.
Processing the report 14.
Processing the report 15.
Processing the report 16.
Processing the report 17.
Processing the report 18.
Processing the report 19.
Processing the report 20.
Processing the report 21.
Processing the report 22.
Processing the report 23.
Processing the report 24.
Processing the report 25.
Processing the report 26.
Processing the report 27.
Processing the report 28.
Processing the report 29.
Processing the report 30.
Processing the report 31.
Processing the report 32.
Processing the report 33.
Processing the report 34.
Processing the report 35.
Processing the report 36.
Processing the report 37.
Processing the report 

KeyboardInterrupt: 

In [9]:
sum(label_out)

549

In [ ]:
rows = zip(pid_out, doc_out, sent_out, label_out)

with open('factuality_train.csv', "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)